# Python 多线程和多进程

本教程将学习Python中的多线程和多进程编程，帮助处理并发任务和提高程序性能。

## 1. 多线程编程 - threading 模块


In [1]:
import threading
import time

# 定义一个简单的线程函数
def print_numbers(thread_name, count):
    """打印数字的函数"""
    for i in range(1, count + 1):
        print(f"{thread_name}: {i}")
        time.sleep(0.1)  # 模拟耗时操作

# 创建线程
thread1 = threading.Thread(target=print_numbers, args=("线程1", 5))
thread2 = threading.Thread(target=print_numbers, args=("线程2", 5))

# 启动线程
thread1.start()
thread2.start()

# 等待线程完成
# join() 等待线程完成
thread1.join()
thread2.join()

print("所有线程执行完成！")


线程1: 1
线程2: 1
线程1: 2
线程2: 2
线程1: 3
线程2: 3
线程1: 4
线程2: 4
线程1: 5
线程2: 5
所有线程执行完成！


In [2]:
# 创建线程类（继承Thread类）
class MyThread(threading.Thread):
    """自定义线程类"""
    
    def __init__(self, thread_name, count):
        super().__init__()
        self.thread_name = thread_name
        self.count = count
    
    def run(self):
        """线程执行的方法"""
        for i in range(1, self.count + 1):
            print(f"{self.thread_name}: {i}")
            time.sleep(0.1)

# 使用自定义线程类
thread1 = MyThread("自定义线程1", 5)
thread2 = MyThread("自定义线程2", 5)

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print("自定义线程执行完成！")


自定义线程1: 1
自定义线程2: 1
自定义线程1: 2
自定义线程2: 2
自定义线程1: 3
自定义线程2: 3
自定义线程2: 4
自定义线程1: 4
自定义线程2: 5
自定义线程1: 5
自定义线程执行完成！


## 2. 线程同步 - Lock（锁）


In [3]:
# 线程同步：使用Lock避免竞态条件
counter = 0
lock = threading.Lock()

def increment_counter():
    """增加计数器（带锁保护）"""
    global counter
    for _ in range(100000):
        lock.acquire()  # 获取锁
        try:
            counter += 1
        finally:
            lock.release()  # 释放锁

# 或者使用with语句（推荐）
def increment_counter_safe():
    """增加计数器（使用with语句）"""
    global counter
    for _ in range(100000):
        with lock:  # 自动获取和释放锁
            counter += 1

# 创建多个线程
threads = []
for i in range(5):
    thread = threading.Thread(target=increment_counter_safe)
    threads.append(thread)
    thread.start()

# 等待所有线程完成
for thread in threads:
    thread.join()

print(f"最终计数器值: {counter}")  # 应该是 500000


最终计数器值: 500000


## 3. 线程池 - ThreadPoolExecutor


In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def task(name, duration):
    """模拟任务"""
    print(f"任务 {name} 开始执行")
    time.sleep(duration)
    print(f"任务 {name} 完成")
    return f"任务 {name} 的结果"

# 使用线程池
with ThreadPoolExecutor(max_workers=3) as executor:
    # 提交任务
    futures = []
    tasks = [("任务1", 1), ("任务2", 2), ("任务3", 1), ("任务4", 1), ("任务5", 2)]
    
    for name, duration in tasks:
        future = executor.submit(task, name, duration)
        futures.append(future)
    
    # 获取结果
    print("\n获取任务结果:")
    for future in as_completed(futures):
        result = future.result()
        print(f"收到结果: {result}")

print("\n所有任务完成！")


任务 任务1 开始执行
任务 任务2 开始执行
任务 任务3 开始执行

获取任务结果:
任务 任务1 完成
任务 任务4 开始执行
收到结果: 任务 任务1 的结果
任务 任务3 完成
任务 任务5 开始执行
收到结果: 任务 任务3 的结果
任务 任务2 完成
任务 任务4 完成
收到结果: 任务 任务2 的结果
收到结果: 任务 任务4 的结果
任务 任务5 完成
收到结果: 任务 任务5 的结果

所有任务完成！


## 4. 多进程编程 - multiprocessing 模块


In [6]:
import multiprocessing
import time

def worker(process_name, count):
    """工作进程函数"""
    print(f"{process_name} 开始工作")
    for i in range(count):
        print(f"{process_name}: {i + 1}")
        time.sleep(0.1)
    print(f"{process_name} 完成工作")

if __name__ == '__main__':  # Windows系统需要这个保护
    # 创建进程
    process1 = multiprocessing.Process(target=worker, args=("进程1", 5))
    process2 = multiprocessing.Process(target=worker, args=("进程2", 5))
    
    # 启动进程
    process1.start()
    process2.start()
    
    # 等待进程完成
    process1.join()
    process2.join()
    
    print("所有进程执行完成！")


所有进程执行完成！


In [7]:
# 进程池 - ProcessPoolExecutor
from concurrent.futures import ProcessPoolExecutor
import os

def cpu_bound_task(n):
    """CPU密集型任务（计算平方和）"""
    result = sum(i * i for i in range(n))
    return result, os.getpid()  # 返回进程ID

if __name__ == '__main__':
    # 使用进程池处理CPU密集型任务
    numbers = [1000000, 1000000, 1000000, 1000000]
    
    print("使用进程池处理CPU密集型任务:")
    with ProcessPoolExecutor(max_workers=4) as executor:
        start_time = time.time()
        results = list(executor.map(cpu_bound_task, numbers))
        end_time = time.time()
    
    for result, pid in results:
        print(f"结果: {result}, 进程ID: {pid}")
    
    print(f"\n总耗时: {end_time - start_time:.2f} 秒")


使用进程池处理CPU密集型任务:


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

## 5. 进程间通信 - Queue


In [8]:
from multiprocessing import Process, Queue
import time

def producer(queue, items):
    """生产者进程：向队列中添加数据"""
    for item in items:
        print(f"生产者: 生产 {item}")
        queue.put(item)
        time.sleep(0.1)
    queue.put(None)  # 发送结束信号
    print("生产者完成")

def consumer(queue):
    """消费者进程：从队列中取出数据"""
    while True:
        item = queue.get()
        if item is None:  # 收到结束信号
            break
        print(f"消费者: 消费 {item}")
        time.sleep(0.1)
    print("消费者完成")

if __name__ == '__main__':
    # 创建队列
    queue = Queue()
    
    # 创建进程
    producer_process = Process(target=producer, args=(queue, [1, 2, 3, 4, 5]))
    consumer_process = Process(target=consumer, args=(queue,))
    
    # 启动进程
    producer_process.start()
    consumer_process.start()
    
    # 等待进程完成
    producer_process.join()
    consumer_process.join()
    
    print("进程间通信完成！")


进程间通信完成！


## 6. 线程 vs 进程：何时使用什么

### 使用多线程的情况：
1. I/O密集型任务（文件读写、网络请求等）
2. 需要共享数据和状态
3. 任务相对轻量级

### 使用多进程的情况：
1. CPU密集型任务（计算、图像处理等）
2. 需要真正的并行执行（绕过GIL）
3. 需要进程隔离（提高稳定性）

### GIL（全局解释器锁）说明：
- Python的GIL限制了同一时刻只能有一个线程执行Python字节码
- 对于CPU密集型任务，多线程不会提高性能，应使用多进程
- 对于I/O密集型任务，多线程仍然有效，因为I/O等待期间会释放GIL
